# Backpropagation Algorithm

In this notebook I implement backpropagation algorithm. See section *C*,  chapter $6$ of this [paper](https://arxiv.org/pdf/1803.08823)
cite [paper](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf)

In this notebook I attempt to reproduce the results of the first expriement done in [learning representations by back-propagating errors ](https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf). 

**Backpropagation Algorithm:**

- [ ] Add the defintions of the weights and the activation ..etc..  
Gradient Decent allows us to identify the set of weights which optimize the cost function of the network 
$$
\omega^{*} = \arg\max\limits_{\omega} \mathcal{C} (y, \hat{y})
$$

However, It is not straightforward to compute the gradient of the cost function $\mathcal{C} (y, \hat{y})$. However, it is possible to exploit the feedforward nature of a neural network to effectively compute the gradient of the cost fucntion. In the coming lines I will present a simple derivations of the four equations that represent the working horse of the Backpropgation algorithm. We start first by defining a mathmatical object that quantifies the senstivety of the cost function to the change in the weighted input of the output layer $L$, $z_j^{L}$. Which can be also viewed as the error of $j$-th neuron in the $L$-th layer, $\delta_j^L$. 

$$
\delta_j^L = \frac{\partial \mathcal{C}}{\partial z_j^L}
$$

In the same manner we can define the error of the $j$-th neuron in the $l$-th layer as 
$$
\begin{equation}
\delta_j^l = \frac{\partial \mathcal{C}}{\partial z_j^l} 
\end{equation}
$$

Using the chain rule we could write the error $\delta_j^l$ as
$$
\begin{equation}
\delta_j^l = \frac{\partial \mathcal{C}}{\partial a_j^l}\frac{\partial a_j^l}{\partial z_j^l} = \frac{\partial \mathcal{C}}{\partial a_j^l}\sigma'(z_j^l)
\end{equation}
$$

$\sigma'(z_j^l)$ corresponds to the dervative of the activation function with respect to the weighted input. The previous equation represents the first of the four backpropgation equations. 

Similarly, we can use the chain rule again and show that the error $\delta_j^l$ could be viewed as the senstivety of the cost function for the changes in the bias  of the $j$-th neuron residing in the $l$-th layer.
$$
\begin{equation}
\delta_j^l = \frac{\partial \mathcal{C}}{\partial b_j^l}\frac{\partial b_j^l}{\partial z_j^l} = \frac{\partial \mathcal{C}}{\partial b_j^l}
\end{equation}
$$
This is the second equation :) 

Since the cost function depends explicitly on the output of the last layer, the error could seen as propgating from the last layer to the inner most layers of the network. 

$$
\begin{align}
\delta_j^l &= \sum\limits_{k}\frac{\partial \mathcal{C}}{\partial z_k^{l+1}}\frac{\partial z_k^{l+1}}{\partial z_j^l}\\
           &= \sum\limits_{k}\delta_j^{l+1}\frac{\partial z_k^{l+1}}{\partial z_j^l}\\
           &= \sum\limits_{k}\left( \delta_j^{l+1}\omega_{kj}^{l+1} \right)\sigma'(z_j^l)
\end{align}
$$

This is the third equation, bare with me we still have one more. 

The last one defines the dervative of the cost function with repsect to weight connecting the $k$-th neuron at layer $l-1$ with the $j$-th neurons at the layer $l$

$$
\frac{\partial \mathcal{C}}{\partial \omega_{jk}^{l}} = \frac{\partial \mathcal{C}}{\partial z_j^{l}}\frac{\partial z_j^{l}}{\partial \omega_{jk}^{l}} = \delta_{j}^{l} a_k^{l-1}
$$

The alogrithm work in the following manner 

1. First compute all the activations of the input layer $a_j^0$ 
2. Feedforward, using the fact that
$$
a_j^l = \sigma\left(\sum\limits_{k} \omega_{jk}^l a_j^{l-1} + b_j^l \right)
$$
Compute the activations of all the subsequent layers
3. Compute the error at the top layer $\delta_j^L$ 
4. Backpropgate the error: use the third equation to compute the errors in the rest of the layers
5. Finally, compute the gradient using the second and fourth equation 

In [1081]:
# I start first by importing necessary libraries 
import numpy as np 
import matplotlib.pyplot as plt 
import random 
import itertools
from tqdm import tqdm 
# Try fixing the precision :) 

In [170]:
# ! pip install tqdm 

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [1082]:
class Network:
    def __init__(self, sizes:list=[6, 2, 1]):
        # self.seed = np.random.randint(0, 100)
        seed = 33 # Seed that will reproduce the results 
        np.random.seed(seed)  # Set seed for reproducibility
        self.n_layers = len(sizes) # Getting the number of layers 
        self.sizes = sizes # Getting the number of neuron per layers 
        
        """
            Intilizing the weights and the biases following a standard normal distrubution 
        """
        self.biases  =  [np.random.randn(y, 1).astype(np.float64) for y in sizes[1:]] 
        # self.biases  =  [np.ones((y, 1)).astype(np.float64) for y in sizes[1:]] 

        self.weights = [np.random.randn(y, x).astype(np.float64) for x, y in zip(sizes[:-1], sizes[1:])]
        
        # self.weights = [np.random.uniform(-3, 3, (y, x)).astype(np.float64) for x, y in zip(sizes[:-1], sizes[1:])]

    def feed_forward(self, a):
        """
        Feed-Forward step 
        """
        for b, w in zip(self.biases, self.weights):
            a = self.sigmoid(np.dot(w, a) + b)
        return a
        
    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        
        if test_data: n_test = len(test_data)
        n = len(training_data) # Number of training points 
        for j in tqdm(range(epochs), desc="Training Epochs", unit=" Epochs"):
            random.shuffle(training_data) # Shuffle the training data 
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches: 
                self.update_mini_batch(mini_batch, eta)
            if test_data: 
                if j % 10 == 0:
                    print(f"Epoch {j} : {self.evaluate(test_data)} /{n_test}")
            # else:
            #     # pass
            #     if j % 10 == 0:
            #         print(f"Epoch {j} complete")
    def GD(self, training_data, epochs, eta, test_data=None):
        if test_data: n_test =len(test_data)
        n = len(training_data)

        for j in tqdm(np.arange(epochs)):
            self.update(training_data, eta)
            # for x in training_data:
            #     self.update(x, eta)

    def update_mini_batch(self, mini_batch, eta):
        """
            Function to update the mini batch 
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases] # Intilizing the gradient with respect to the biases 
        nabla_w = [np.zeros(w.shape) for w in self.weights] # Intilizing the gradient with respect the weights 
        
        for x, y in mini_batch:
            
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        self.weights = [w - (eta/len(mini_batch)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta/len(mini_batch)) * nb for b, nb in zip(self.biases, nabla_b)]
    
    def update(self, training_data, eta):
        """
        Function to update weights and biases using full batch gradient descent
        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        for x, y in training_data:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        
        self.weights = [w - (eta / len(training_data)) * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b - (eta / len(training_data)) * nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]  # Gradient w.r.t. biases
        nabla_w = [np.zeros(w.shape) for w in self.weights]  # Gradient w.r.t. weights
    
        # Feedforward
        activation = x
        activations = [x]
        zs = []
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)  # Fixed issue here
            activation = self.sigmoid(z)
            activations.append(activation)
    
        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * self.dsigmoid(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].T)
    
        for l in range(2, self.n_layers):
            z = zs[-l]
            sp = self.dsigmoid(z)
            delta = np.dot(self.weights[-l+1].T, delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].T)
    
        return (nabla_b, nabla_w)
        
    def evaluate(self, test_data):
        test_results = [np.argmax(self.feed_forward(x), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)
    def cost_derivative(self, output, y):
        return (output-y)
        
    def sigmoid(self, x): 
        """
            Activation function 
        """
        return 1.0 / (1.0 + np.exp(-x))
    def dsigmoid(self, x): 
        """
        Derivative of the activation function with respect to the input 
        """
        return self.sigmoid(x) * (1.0 - self.sigmoid(x))
    def predict(self, x):
        output = self.feed_forward(x)
        return output

## Application

The rest of the notebook will attempt to repoduce the results of the paper learning representations with backpropgation algorithm

In [1178]:
  

# An attempt to reproduce the results of https://www.iro.umontreal.ca/~vincentp/ift3395/lectures/backprop_old.pdf
# Generating the data 
def is_symmetric(seq):
    return seq == seq[::-1]
m = 6
# Generate all 6-bit sequences and labels
all_sequences = list(itertools.product([0, 1], repeat=m))
labels = [1 if is_symmetric(seq) else 0 for seq in all_sequences]
training_data = [(np.array(seq, dtype=np.float32).reshape(m, 1), 
                  np.array([label], dtype=np.float32).reshape(1, 1)) 
                 for seq, label in zip(all_sequences, labels)]

In [1179]:
print(f"There are: {len(training_data)} datapoints")

There are: 64 datapoints


In [665]:
# train_size = int(0.8 * len(training_data))
# train_set = training_data[:train_size]
# test_set = training_data[train_size:]

In [1180]:
# Defining neural network 
sizes = [m, 2, 1]

In [1181]:
net = Network(sizes)
eta = 1.0
epochs = 500
batch_size = 1
net.SGD(training_data ,epochs, batch_size, eta, test_data=None)
print("The weights of the first hidden layer")
print(net.weights[0].transpose().round(1))
print("The biases of the hidden layer")
print(net.biases[0].transpose().round(1))
print("The bias of the output layer")
print(net.biases[1].transpose().round(1))

Training Epochs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [00:02<00:00, 198.08 Epochs/s]

The weights of the first hidden layer
[[-11.2  11.3]
 [  2.8  -2.9]
 [  5.6  -5.7]
 [ -5.6   5.7]
 [ -2.8   2.9]
 [ 11.2 -11.3]]
The biases of the hidden layer
[[-2. -2.]]
The bias of the output layer
[[5.5]]


# Remarks 
1. Using $\varepsilon=0.9$ with `batch_size=6` and `Epochs=1425`, and weights and biases as standard normal distros it is possible to generate the alternating sign effect
2. epoch = 5